#### Собрать более-менее приличную рекомендательную систему с помощью GBDT. Использование трюков из лекции (и не из неё), тюнинг и разумных объемов фича инжиниринг прям нужен. 

In [42]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import ShuffleSplit, cross_validate, train_test_split

from catboost import CatBoostRegressor
from catboost import Pool
from catboost import cv

from gensim.models import Word2Vec

Загрузим юзеров, выкинем странные признаки, посчитаем сколько они зарегестрированы в приложении 

In [2]:
users = pd.read_csv('dataset/members.csv'
                    , dtype={
                        'msno': 'category'
                        , 'bd': 'int'
                        , 'city': 'category'
                        , 'gender': 'category'
                        , 'registration_init_time': str
                        , 'expiration_date': str
                    }
                    , parse_dates=['registration_init_time', 'expiration_date'])

users['time'] = users['expiration_date'] - users['registration_init_time']
users['gender'] = users['gender'].cat.add_categories('-').fillna('-')

users.drop(['registered_via', 'expiration_date', 'registration_init_time'], axis=1, inplace=True)

users.head(3)

,msno,city,bd,gender,time
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,-,2223 days
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,-,725 days
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,-,457 days


Загрузим песни  
В случае нескольких жанров, оставим первый

In [3]:
types = {
    'song_id': 'category'
    , 'artist_name': 'category'
    , 'composer': 'category'
    , 'language': 'category'
    , 'name': 'category'
    , 'isrc': 'str'
}

info = pd.read_csv('dataset/song_extra_info.csv', dtype=types)
songs = pd.read_csv('dataset/songs.csv', dtype=types).merge(info, how='left')

def take_first(some):
    return some.split('|')[0]

songs['artist_name'] = songs['artist_name'].cat.add_categories('-').fillna('-').agg(take_first)
songs['composer'] = songs['composer'].fillna('-').agg(take_first)
songs['genre'] = songs['genre_ids'].fillna('-').agg(take_first)
songs.drop('lyricist', axis=1, inplace=True)

Раскодируем язык, лейбл и год песни  

In [4]:
def year_to_int(year):
    if year is None:
        return None
    if year > 21:
        return 1900 + year
    return 2000 + year

year_to_int(68), year_to_int(int('08'))

(1968, 2008)

In [5]:
songs['country'] = songs['isrc'].agg(lambda s: str(s)[:2] if str(s).lower() != 'nan' else '-').astype('category')
songs['label'] = songs['isrc'].agg(lambda s: str(s)[2:5] if str(s).lower() != 'nan' else '-').astype('category')
songs['year'] = songs['isrc'].agg(lambda s: int(str(s)[5:7]) if str(s).lower() != 'nan' else None)

songs['year'] = songs['year'].agg(lambda s: year_to_int(s))
songs['year'] = songs['year'].fillna(songs['year'].median()).astype('int')

songs.drop('isrc', axis=1, inplace=True)

songs.head()

,song_id,song_length,genre_ids,artist_name,composer,language,name,genre,country,label,year
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,3.0,焚情,465,TW,B53,2014
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY,31.0,PLAYING WITH FIRE,444,-,-,2011
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,-,31.0,SORRY| SORRY,465,-,-,2011
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,3.0,愛我的資格,465,TW,C95,2002
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,52.0,Mary Had a Little Lamb,726,-,-,2011


Всё кроме юзеров склеим в один датасет  
Позволю себе выкинуть песни, про которые нет информации, из-за того что, кто-то в неправильном порядке выгрузил таблицы из базы данных  

In [6]:
df = pd.read_csv('dataset/train.csv', dtype={
    'msno': 'category'
    , 'song_id': 'category'
    , 'source_system_tab': 'category'
    , 'source_screen_name': 'category'
    , 'source_type': 'category'
    , 'target': 'int'}).merge(songs, how='left')

df.drop(['source_system_tab', 'source_screen_name', 'genre_ids'], axis=1, inplace=True)
df['source_type'] = df['source_type'].cat.add_categories('-').fillna('-')

df.dropna(inplace=True)

df.head()

,msno,song_id,source_type,target,song_length,artist_name,composer,language,name,genre,country,label,year
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,online-playlist,1,206471.0,Bastille,Dan Smith,52.0,Good Grief,359,GB,UM7,2016.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,local-playlist,1,284584.0,Various Artists,-,52.0,Lords of Cardboard,1259,US,3C6,1999.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,local-playlist,1,225396.0,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,52.0,Hip Hop Is Dead(Album Version (Edited)),1259,US,UM7,2006.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,local-playlist,1,255512.0,Soundway,Kwadwo Donkoh,-1.0,Disco Africa,1019,GB,UQH,2010.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,online-playlist,1,187802.0,Brett Young,Brett Young,52.0,Sleep Without You,1011,QM,3E2,2016.0


Для каждого юзера посчитаем его предпочтения по жанрам

In [41]:
%%time

u = users['msno'].to_numpy()
msno_to_index = {}
for i in range(len(u)):
    msno_to_index[u[i]] = i
    
g = df[df['target'] == 1]['genre'].unique()
genre_to_index = {}
for i in range(len(g)):
    genre_to_index[g[i]] = i

counts = np.zeros((len(u), len(g)), dtype=np.float16)
    
likes = df[df['target'] == 1][['genre', 'msno']].to_numpy()
for genre, msno in likes:
    msno_index = msno_to_index[msno]
    genre_index = genre_to_index[genre]
    counts[msno_index][genre_index] += 1
    
counts = np.apply_along_axis(lambda a: a / len(a), 1, counts)

for genre in g:
    users[genre] = counts.T[genre_to_index[genre]]
    users[genre] = users[genre].astype(np.float16)
    
users.head()

Wall time: 21 s


,msno,city,bd,gender,time,359,1259,1019,1011,465,...,808,1117,1162,2144,2127,95,2008,1266,1110,2065
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,-,2223 days,0.0,0.007042,0.0,0.007042,2.992188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,-,725 days,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,-,457 days,0.0,0.000000,0.0,0.000000,0.014084,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,-,1 days,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,-,138 days,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Теперь юзеров можно добавить в основной датасет

In [8]:
df = df.merge(users, how='left')
df

,msno,song_id,source_type,target,song_length,artist_name,composer,language,name,genre,...,808,1117,1162,2144,2127,95,2008,1266,1110,2065
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,online-playlist,1,206471.0,Bastille,Dan Smith,52.0,Good Grief,359,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,local-playlist,1,284584.0,Various Artists,-,52.0,Lords of Cardboard,1259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,local-playlist,1,225396.0,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,52.0,Hip Hop Is Dead(Album Version (Edited)),1259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,local-playlist,1,255512.0,Soundway,Kwadwo Donkoh,-1.0,Disco Africa,1019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,online-playlist,1,187802.0,Brett Young,Brett Young,52.0,Sleep Without You,1011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375806,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,local-playlist,1,267958.0,Dr. Dre,Andre R. Young,52.0,Still D.R.E.,1259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7375807,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,song,0,227404.0,R. City feat. Adam Levine,Theron Thomas,52.0,Locked Away,465,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7375808,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,song,1,258298.0,兄弟本色G.U.T.S. (姚中仁、張震嶽、頑童MJ116),姚中仁 MC HotDog 陳昱榕 Ｅ-SO 周文傑 KENZY 林睦淵,3.0,FLY OUT,465,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7375809,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,online-playlist,1,524146.0,Brad Mehldau Trio,-,52.0,Wonderwall,2122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Поделим данные

In [9]:
x = df.drop('target', axis=1)
y = df['target']

Не знаю, считается ли это за хороший тюнинг  
Но catboost отлично автоматически настраивает гиперпараметры, если поставить побольше итераций 

In [12]:
%%time
gbdt = cross_validate(CatBoostRegressor(iterations=200
                        , learning_rate=0.5
                        , task_type="CPU"
                        , custom_metric=['AUC:hints=skip_train~false']
                        , cat_features=['msno','song_id','source_type','genre','artist_name','composer'
                                        ,'language','name','country','city','bd','gender','label'])
                    , x, y, cv=ShuffleSplit(n_splits=1, test_size=0.2)
                    , return_estimator=True,return_train_score=True, scoring="roc_auc")

0:	learn: 0.4574468	total: 18s	remaining: 59m 39s
1:	learn: 0.4398068	total: 31.9s	remaining: 52m 35s
2:	learn: 0.4294481	total: 46s	remaining: 50m 23s
3:	learn: 0.4250410	total: 54.9s	remaining: 44m 49s
4:	learn: 0.4238004	total: 1m 5s	remaining: 42m 50s
5:	learn: 0.4225890	total: 1m 17s	remaining: 41m 42s
6:	learn: 0.4215635	total: 1m 31s	remaining: 42m 16s
7:	learn: 0.4210788	total: 1m 45s	remaining: 42m
8:	learn: 0.4200037	total: 2m 2s	remaining: 43m 15s
9:	learn: 0.4195602	total: 2m 14s	remaining: 42m 30s
10:	learn: 0.4192450	total: 2m 26s	remaining: 42m
11:	learn: 0.4185825	total: 2m 42s	remaining: 42m 26s
12:	learn: 0.4181370	total: 2m 56s	remaining: 42m 13s
13:	learn: 0.4177858	total: 3m 9s	remaining: 41m 52s
14:	learn: 0.4173301	total: 3m 20s	remaining: 41m 7s
15:	learn: 0.4169704	total: 3m 32s	remaining: 40m 46s
16:	learn: 0.4167169	total: 3m 46s	remaining: 40m 40s
17:	learn: 0.4165582	total: 3m 57s	remaining: 39m 58s
18:	learn: 0.4162732	total: 4m 8s	remaining: 39m 25s
19:	l

151:	learn: 0.4069323	total: 32m 24s	remaining: 10m 14s
152:	learn: 0.4069063	total: 32m 35s	remaining: 10m
153:	learn: 0.4068794	total: 32m 47s	remaining: 9m 47s
154:	learn: 0.4068681	total: 32m 59s	remaining: 9m 34s
155:	learn: 0.4068483	total: 33m 9s	remaining: 9m 21s
156:	learn: 0.4068265	total: 33m 23s	remaining: 9m 8s
157:	learn: 0.4067448	total: 33m 34s	remaining: 8m 55s
158:	learn: 0.4067170	total: 33m 46s	remaining: 8m 42s
159:	learn: 0.4066661	total: 33m 57s	remaining: 8m 29s
160:	learn: 0.4066420	total: 34m 6s	remaining: 8m 15s
161:	learn: 0.4066087	total: 34m 19s	remaining: 8m 3s
162:	learn: 0.4065858	total: 34m 29s	remaining: 7m 49s
163:	learn: 0.4065410	total: 34m 40s	remaining: 7m 36s
164:	learn: 0.4065274	total: 34m 52s	remaining: 7m 23s
165:	learn: 0.4065090	total: 35m 6s	remaining: 7m 11s
166:	learn: 0.4064820	total: 35m 18s	remaining: 6m 58s
167:	learn: 0.4064282	total: 35m 28s	remaining: 6m 45s
168:	learn: 0.4063904	total: 35m 40s	remaining: 6m 32s
169:	learn: 0.406

In [13]:
gbdt

{'fit_time': array([2793.96203351]),
 'score_time': array([462.37457943]),
 'estimator': (<catboost.core.CatBoostRegressor at 0x1fdc095ef48>,),
 'test_score': array([0.8531347]),
 'train_score': array([0.93453449])}

Немного переобучились, но результаты отличные  
**train - 0.93**  
**test  - 0.85**

#### Пользуясь информацией о треках построить неклассические эмбеддинги и попробовать использовать их:

#### а) показать симилары глазами

Поделим данные

In [15]:
train = df[df['target'] == 1].head(int(len(df) * 0.8))
test = df.tail(int(len(df) * 0.2))

Для w2v посчитаем контексты - что лайкал юзер

In [17]:
%%time
from collections import defaultdict

contexts = defaultdict(list)

for msno, song_id in train[['msno', 'song_id']].sort_values(by='msno').to_numpy():
    contexts[msno].append(song_id)

Wall time: 7.62 s


In [37]:
%%time
w2v = Word2Vec(contexts.values(), window=10, sg=1, hs=0, vector_size=70, workers=8
               , negative=5, alpha=0.03, min_alpha=0.0007, min_count=10)

Wall time: 1min 5s


Мою музыку не слушают, поэтому будем смотреть на Lady Gaga

In [50]:
def index_to_song_name(w2v, song_id):
    idx = w2v.wv.key_to_index[song_id]
    similars = w2v.wv.most_similar(idx)
    for sim, _ in similars:
        print(songs[songs['song_id'] == sim]['artist_name'].tolist())

index_to_song_name(w2v, 'eW1d3l1sSncU4sGM69MJtIG79wnXbdy8FZKpFVElS0g=')

['Lady Gaga']
['Sia']
['P!nk']
['Shawn Mendes']
['Lukas Graham']
['gnash']
['Bruno Mars']
['Sia']
['Colbie Caillat']
['The Chainsmokers']


Симилары не слишком радуют, так как всего одна песня того же исполнителя (это не та же самая)

In [ ]:
# %%time

# user_embedings = {}
# i = 0
# for msno, song_ids in contexts.items():
#     song_embedings = []
#     for song_id in song_ids:
#         if song_id in w2v.wv.key_to_index.keys():
#             song_embedings.append(w2v.wv[song_id])
#         else:
#             song_embedings.append(np.zeros(70))
#     song_embeding = np.asarray(song_embedings)
#     user_embeding = song_embedings.mean(axis=0)
#     user_embedings[msno] = user_embeding
#     i += 1
#     if i == 100:
#         break